In [ ]:
"""
Created on Wed Feb 01 16:21 2023

Check the input masks (grounding line, ice front, ice shelves, ...)

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import matplotlib as mpl
from tqdm.notebook import tqdm
import basal_melt_param.plume_functions as pf

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'bi646'
map_lim = [-3000000,3000000]


In [ ]:
outputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
outputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
inputpath_raw = '/bettik/burgardc/DATA/NN_PARAM/raw/'

In [ ]:
file_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo_clean.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_mask_cut = uf.cut_domain_stereo(file_mask, map_lim, map_lim)
file_mask_cut = file_mask_cut.assign_coords({'time': range(len(file_mask.time))})

file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_2000.nc')
file_TS_cut = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim)

In [ ]:
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)
file_conc_cut = file_conc_cut.assign_coords({'time': file_mask_cut.time})

In [ ]:
file_conc_cut['isfdraft_conc'].isel(time=30).where((file_TS_cut['so'].max('deptht') > 0), 0).plot()

In [ ]:
file_mask_cut['ls_mask012'].isel(time=30).plot()

In [ ]:
file_mask_cut['ls_mask012'].isel(time=30).where((file_TS_cut['so'].max('deptht') > 0), 2).plot() # there are a few ice shelf patches inland

In [ ]:
file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_2000.nc')

In [ ]:
file_TS_orig['so'].max('deptht').plot(cmap=mpl.cm.Reds_r)

In [ ]:
file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy54.nc')

In [ ]:
file_isf['ground_mask'].plot()

In [ ]:
file_isf['ISF_mask'].plot(vmin=0,vmax=10)

In [ ]:
file_isf['IF_mask'].plot(vmax=10)

In [ ]:
#file_isf['GL_mask'].where(file_isf['GL_mask'] == 66, drop=True).plot()
file_isf['IF_mask'].where(file_isf['IF_mask'] == 66, drop=True).plot()

In [ ]:
file_isf['dIF'].plot(vmax=100000)

In [ ]:
box_2D = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_yy35.nc')

In [ ]:
box_2D['box_location'].sel(box_nb_tot=5).plot()

In [ ]:
file_isf_00 = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy00.nc')

In [ ]:
file_mask_54 = file_mask_cut.isel(time=54)

In [ ]:
isf_mask_default = file_mask_54['ls_mask012'].where(file_isf_00['ISF_mask'] < 3).combine_first(file_isf_00['ISF_mask']).plot(vmax=3)

In [ ]:
def def_ground_mask(file_msk, dist, add_fac):

    """
    Define a mask for the Antarctic continent as such (not the islands). 
    
    This function defines the points that are part of the Antarctic continent as such (not the islands). 
    
    Parameters
    ----------
    file_msk : xr.DataArray
        Mask separating ocean (0), ice shelves (between 0 and 2, excluding 0 and 2), grounded ice (2)
    dist : int
        Defines the size of the starting square - should be small if the resolution is coarse and high if the resolution is fine. Default is currently 150 but you can play around. A good indicator to see if it is too high is if you see the small upper tail of the Ross ice shelf or if it is masked as ground.
    add_fac : int
       Defines additional iterations. Was introduced to get to the end of the Antarctic Peninsula, sometimes it would not get there otherwise. Current default is 100 but you are welcome to play around with it.
    Returns
    -------
    mask_ground : xr.DataArray
        Array showing the coverage of the Antarctic continent (0 for islands, 1 for ocean and ice shelves, 2 for mainland)
    """  
    
    mask_10 = file_msk.where(file_msk == 2, 0).where(file_msk != 2,1) #set all ice shelves and open ocean to 0, set all grounded ice to 1

    mask_gnd = mask_10.where(mask_10>0, drop=True)
    mask_gnd = mask_gnd.where(mask_gnd>0,0)
    
    meshx_gnd, meshy_gnd = np.meshgrid(mask_gnd.x,mask_gnd.y)
    meshx_gnd_da = mask_gnd.copy(data=np.broadcast_to(meshx_gnd, mask_gnd.shape))
    meshy_gnd_da = mask_gnd.copy(data=np.broadcast_to(meshy_gnd, mask_gnd.shape))
    
    dx = abs(meshx_gnd_da.x[2] - meshx_gnd_da.x[1])
    dy = abs(meshx_gnd_da.y[2] - meshx_gnd_da.y[1])
    
    max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
    half_range = round(max_len_xy/2)
    
    mask_core = mask_gnd.where(~((uf.in_range(meshx_gnd_da, [-2*dist*dx,2*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-2*dist*dy,2*dist*dy]))), 5)
    
    # filter that checks the point around
    weights_filter = np.zeros((3,3))
    weights_filter[0,1] = 1
    weights_filter[1,0] = 1
    weights_filter[1,2] = 1
    weights_filter[2,1] = 1
    
    weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])
    
    if 'time' in mask_gnd.dims:

        iter_list = []
        for tt,timet in enumerate(tqdm(mask_gnd.time)):

            mask_core_tt = mask_core.isel(time=tt)
            iter_mask = mask_core_tt.copy()
            for n in range(half_range+2*dist+add_fac):
                corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
                iter_mask = iter_mask.where(~((corr >= 5) & (mask_core_tt == 1)),5)

            iter_list.append(iter_mask)

        iter_mask_all = xr.concat(iter_list,dim='time')
        iter_mask_all = iter_mask_all.assign_coords({'time': mask_gnd.time})

        mask_ground = iter_mask_all.where(iter_mask_all !=5, 2).reindex_like(mask_10)
    
    else:
        
        iter_mask = mask_core.copy()
        for n in tqdm(range(half_range+2*dist+add_fac)):
            corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
            iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

        mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)
        
    mask_ground = mask_ground.where(mask_ground>0,0)
    
    return mask_ground

In [ ]:
file_isf_00['ISF_mask'].plot()

In [ ]:
FRIS_00 = file_conc_cut['isfdraft_conc'].isel(time=0).where(file_isf_00['ISF_mask'] == 11, drop=True)
FRIS_00_test = file_conc_cut['isfdraft_conc'].isel(time=0).where(file_isf_00['ISF_mask'] == 11, drop=True) * 0 + 1
FRIS_00_test = FRIS_00_test.where(np.isfinite(FRIS_00_test),0)

In [ ]:
FRIS_00_test.plot()

In [ ]:
### THIS IS THE ROUTINE THAT NEEDS TO BE IMPLEMENTED IN MASK SCRIPTS => WORKS WELL
dist = 1
dist_2 = 4
add_fac = 100
tt = 0

file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc').drop('time')
isf_conc_tt = file_conc_cut['isfdraft_conc'].sel(time=tt)
isf_mask = file_isf['ISF_mask']
ground_mask = file_isf['ground_mask']
seeded_isf_mask = isf_mask.copy() * np.nan

for kisf in tqdm(file_isf.Nisf.where(file_isf.Nisf >= 10, drop=True)):
    
    seed_mask = isf_conc_tt.where(isf_mask == kisf, drop=True) * 0 + 1
    seed_mask = seed_mask.where(np.isfinite(seed_mask),0)
    
    meshx_gnd, meshy_gnd = np.meshgrid(seed_mask.x,seed_mask.y)
    meshx_gnd_da = seed_mask.copy(data=np.broadcast_to(meshx_gnd, seed_mask.shape))
    meshy_gnd_da = seed_mask.copy(data=np.broadcast_to(meshy_gnd, seed_mask.shape))
    
    if ((len(meshx_gnd_da.x) > 3) and (len(meshx_gnd_da.y) > 3)) :
        dx = abs(meshx_gnd_da.x[1] - meshx_gnd_da.x[0])
        dy = abs(meshx_gnd_da.y[1] - meshx_gnd_da.y[0])

        center_x = (meshx_gnd_da.x.max() + meshx_gnd_da.x.min())/2
        center_y = (meshx_gnd_da.y.max() + meshx_gnd_da.y.min())/2

        max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
        half_range = round(max_len_xy/2)

        mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                              & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))

        if seed_mask.where(mask_core_range).max() < 1: # if the ice shelf is not in the middle of the cell
            n = 0
            while seed_mask.where(mask_core_range).max() < 1:
                #print(seed_mask.where(mask_core_range).max().values)
                center_x = meshx_gnd_da.x.min() + n*dx
                mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                        & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
                n = n+1


        mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                              & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)

        # filter that checks the point around
        weights_filter = np.zeros((3,3))
        weights_filter[0,1] = 1
        weights_filter[1,0] = 1
        weights_filter[1,2] = 1
        weights_filter[2,1] = 1

        weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

        iter_mask = mask_core.copy()
        for n in range(half_range+2*dist+add_fac):
            corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
            iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

        iter_mask = iter_mask.where(iter_mask == 5)
        iter_mask = iter_mask * 0 + kisf

        kisf_reg_grid = iter_mask.reindex_like(isf_mask)    
        seeded_isf_mask = seeded_isf_mask.combine_first(kisf_reg_grid)

new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 0))
new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)
#mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)

#mask_ground = mask_ground.where(mask_ground>0,0)

In [ ]:
mask_33 = new_isf_mask.copy()

In [ ]:
mask_33.plot()

In [ ]:
### THIS IS THE ROUTINE THAT NEEDS TO BE IMPLEMENTED IN MASK SCRIPTS => WORKS WELL
dist = 1
dist2 = 10
add_fac = 100
tt = 34
kisf = 53

file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc').drop('time')
isf_conc_tt = file_conc_cut['isfdraft_conc'].sel(time=tt)
isf_mask_prev = mask_33
isf_mask = file_isf['ISF_mask']
ground_mask = file_isf['ground_mask']
seeded_isf_mask = isf_mask.copy() * np.nan

#for kisf in tqdm(file_isf.Nisf.where(file_isf.Nisf >= 10, drop=True)):

if kisf == 53:
    dist2 = 100

if (isf_mask_prev == kisf).sum() > 1:
    isf_domain = isf_conc_tt.where(isf_mask_prev == kisf, drop=True) * 0 + 1
elif (isf_mask == kisf).sum() > 1:
    isf_domain = isf_conc_tt.where(isf_mask == kisf, drop=True) * 0 + 1
isf_domain = isf_domain.where(np.isfinite(isf_domain),0)


if ((len(isf_domain.x) > 3) and (len(isf_domain.y) > 3)) :

    dx = abs(isf_domain.x[1] - isf_domain.x[0])
    dy = abs(isf_domain.y[1] - isf_domain.y[0])

    meshx_restric, meshy_restric = np.meshgrid(isf_domain.x,isf_domain.y)
    meshx_restric_da = isf_domain.copy(data=np.broadcast_to(meshx_restric, isf_domain.shape))
    meshy_restric_da = isf_domain.copy(data=np.broadcast_to(meshy_restric, isf_domain.shape))

    center_x = (meshx_restric_da.x.max() + meshx_restric_da.x.min())/2
    center_y = (meshy_restric_da.y.max() + meshy_restric_da.y.min())/2

    max_len_xy = max(len(meshx_restric_da.x),len(meshy_restric_da.y))
    half_range = round(max_len_xy/2)

    mask_core_range = ((uf.in_range(meshx_restric_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                          & uf.in_range(meshy_restric_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))

    if isf_domain.where(mask_core_range).max() < 1: # if the ice shelf is not in the middle of the cell
        n = 0
        while isf_domain.where(mask_core_range).max() < 1:
            #print(center_x)
            #print(seed_mask.where(mask_core_range).max().values)
            center_x = isf_domain.x.min() + n*dx
            mask_core_range = ((uf.in_range(meshx_restric_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                    & uf.in_range(meshy_restric_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
            n = n+1

    # enlarge the domain
    if isf_domain.x[0] < isf_domain.x[1]:
        xxrange = np.arange(isf_domain.x.min() - dist2*dx, isf_domain.x.max() + dist2*dx,dx)
    else:
        xxrange = np.arange(isf_domain.x.max() + dist2*dx, isf_domain.x.min() - dist2*dx,-dx)

    if isf_domain.y[0] < isf_domain.y[1]:
        yyrange = np.arange(isf_domain.y.min() - dist2*dy, isf_domain.y.max() + dist2*dy,dy)
    else:
        yyrange = np.arange(isf_domain.y.max() + dist2*dy, isf_domain.y.min() - dist2*dy,-dy)

    new_domain = isf_conc_tt.sel(x=xxrange,y=yyrange) 
    seed_mask = isf_conc_tt.where(new_domain, drop=True) * 0 + 1
    seed_mask = seed_mask.where(np.isfinite(seed_mask),0)

    meshx_gnd, meshy_gnd = np.meshgrid(seed_mask.x,seed_mask.y)
    meshx_gnd_da = seed_mask.copy(data=np.broadcast_to(meshx_gnd, seed_mask.shape))
    meshy_gnd_da = seed_mask.copy(data=np.broadcast_to(meshy_gnd, seed_mask.shape))


    mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)

    # filter that checks the point around
    weights_filter = np.zeros((3,3))
    weights_filter[0,1] = 1
    weights_filter[1,0] = 1
    weights_filter[1,2] = 1
    weights_filter[2,1] = 1

    weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

    iter_mask = mask_core.copy()
    for n in range(half_range+2*dist+add_fac):
        corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
        iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

    iter_mask = iter_mask.where(iter_mask == 5)
    iter_mask = iter_mask * 0 + kisf

    kisf_reg_grid = iter_mask.reindex_like(isf_mask)    
    seeded_isf_mask = seeded_isf_mask.combine_first(kisf_reg_grid)

new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 0))
new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)
#mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)

#mask_ground = mask_ground.where(mask_ground>0,0)

In [ ]:
iter_mask.plot()

In [ ]:
isf_conc_tt.where(file_isf['ISF_mask']==53).plot()

In [ ]:
new_isf_mask.plot()#.plot(vmin=25,vmax=27)

In [ ]:
isf_conc_tt.where(new_isf_mask < 1).plot()

In [ ]:
file_isf['Nisf'].where(file_isf['isf_name'] == 'Cosgrove')

In [ ]:
meshy_restric_da.plot()

In [ ]:
iter_mask.plot()

In [ ]:
seed_mask.plot()

In [ ]:
mask_core_range.plot()

In [ ]:
isf_domain.plot()

In [ ]:
meshx_restric_da.plot()

In [ ]:
mask_core.plot()

In [ ]:
isf_domain.x.min() + n*dx

In [ ]:
mask_33.plot()

In [ ]:
new_isf_mask.plot(vmin=24,vmax=26)

In [ ]:
file_isf['isf_name'].sel(Nisf=25)

In [ ]:
isf_conc_tt.where(new_isf_mask==0).plot()

In [ ]:
drainage_basins_file = xr.open_dataset(inputpath_raw + 'IceBoundaries_Y2015-2017_Antarctica_v2.2.1.nc')
drainage_basins = drainage_basins_file['Ice_Shelf_Basins_ID'].sel(x=isf_conc_tt.x,y=isf_conc_tt.y)

In [ ]:
drainage_basins.where(isf_conc_tt>0).plot()

In [ ]:
isf_conc_tt.where(np.isnan(drainage_basins)).plot()

In [ ]:
isf_domain.x.max()

In [ ]:
isf_domain.x.min()

In [ ]:
xxrange

In [ ]:
seed_mask.x

In [ ]:
plt.figure()
isf_mask_prev.where(isf_mask_prev == 38).plot()

In [ ]:
seed_mask = isf_conc_tt.sel(x=xxrange,y=yyrange).plot()

In [ ]:
iter_mask.plot()

In [ ]:
seed_mask.plot()

In [ ]:
xxrange

In [ ]:
mask_33.plot(vmin=74,vmax=76)

In [ ]:
file_isf['ISF_mask'].where(file_isf['ISF_mask']==80).plot()

In [ ]:
mask_33 = new_isf_mask.copy()

In [ ]:
isf_conc_tt.where(new_isf_mask<2).plot()

In [ ]:
seed_mask = isf_conc_tt.where(isf_mask == 10, drop=True) * 0 + 1

In [ ]:
file_isf['ISF_mask'].where(file_isf['ISF_mask']==75).plot()

In [ ]:
new_isf_mask.plot()

In [ ]:
plt.figure()
new_isf_mask.where(new_isf_mask==10).plot()

In [ ]:
(new_isf_mask - file_isf['ISF_mask']).plot()

In [ ]:
plt.figure()
file_isf['ISF_mask'].plot()

In [ ]:
file_isf['isf_name'].sel(Nisf=23)

In [ ]:
center_x = (meshx_gnd_da.x.max() + meshx_gnd_da.x.min())/2
    
mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))

In [ ]:
seed_mask.plot()

In [ ]:
### THIS IS TO TEST FOR REMOVING
dist = 1
add_fac = 100
tt = 30

file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc').drop('time')
isf_conc_tt = file_conc_cut['isfdraft_conc'].sel(time=tt)
isf_mask = file_isf['ISF_mask']
ground_mask = file_isf['ground_mask']
seeded_isf_mask = isf_mask.copy() * np.nan

kisf = 25
seed_mask = isf_conc_tt.where(isf_mask == kisf, drop=True) * 0 + 1
seed_mask = seed_mask.where(np.isfinite(seed_mask),0)

meshx_gnd, meshy_gnd = np.meshgrid(seed_mask.x,seed_mask.y)
meshx_gnd_da = seed_mask.copy(data=np.broadcast_to(meshx_gnd, seed_mask.shape))
meshy_gnd_da = seed_mask.copy(data=np.broadcast_to(meshy_gnd, seed_mask.shape))

if ((len(meshx_gnd_da.x) > 3) and (len(meshx_gnd_da.y) > 3)) :
    dx = abs(meshx_gnd_da.x[1] - meshx_gnd_da.x[0])
    dy = abs(meshx_gnd_da.y[1] - meshx_gnd_da.y[0])

    center_x = (meshx_gnd_da.x.max() + meshx_gnd_da.x.min())/2
    center_y = (meshx_gnd_da.y.max() + meshx_gnd_da.y.min())/2

    max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
    half_range = round(max_len_xy/2)

#    mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
#                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)
    
    mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
    
    if seed_mask.where(mask_core_range).max() < 1: # if the ice shelf is not in the middle of the cell
        n = 0
        while seed_mask.where(mask_core_range).max() < 1:
            print(seed_mask.where(mask_core_range).max().values)
            center_x = meshx_gnd_da.x.min() + n*dx
            mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                    & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
            n = n+1

        
#    if seed_mask.sel(x=center_x,y=center_y) < 2:
#        center_x = 0
#    while seed_mask.sel(x=center_x+dx,y=center_y+dy) < 2:
#          center_x = center_x + 1
    mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)

    # filter that checks the point around
    weights_filter = np.zeros((3,3))
    weights_filter[0,1] = 1
    weights_filter[1,0] = 1
    weights_filter[1,2] = 1
    weights_filter[2,1] = 1

    weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

    iter_mask = mask_core.copy()
    for n in range(half_range+2*dist+add_fac):
        corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
        iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

    iter_mask = iter_mask.where(iter_mask == 5)
    iter_mask = iter_mask * 0 + kisf

    #kisf_reg_grid = iter_mask.reindex_like(isf_mask)    
    #seeded_isf_mask = seeded_isf_mask.combine_first(kisf_reg_grid)

#new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 0))
#new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)
#mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)

#mask_ground = mask_ground.where(mask_ground>0,0)

In [ ]:
seed_mask.where(((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))).plot()

In [ ]:
plt.figure()
seed_mask.plot()

In [ ]:
plt.figure()
mask_core.plot()

In [ ]:
iter_mask.plot()

In [ ]:
ls_mask = file_mask_cut['ls_mask012'].sel(time=tt)

In [ ]:
new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 1))
new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)

In [ ]:
new_isf_mask.plot()

In [ ]:
ls_mask.where(file_TS_cut['so'].max() > 0).plot()

In [ ]:
plt.figure()
iter_mask.plot()

In [ ]:
iter_mask.plot()

In [ ]:
plt.figure()
FRIS_00_test.plot()

In [ ]:
FRIS_00.where(np.isfinite(FRIS_00), 0).plot()

In [ ]:
iter_mask.where(iter_mask == 5).plot()

In [ ]:
(((corr >= 5) & (mask_core == 1)) ^ (corr >= 5)).plot()

In [ ]:
iter_mask.where(iter_mask !=5, 2).plot()

In [ ]:
corr.plot()

In [ ]:
iter_mask.

In [ ]:
iter_mask.where((corr >= 5)).plot()

In [ ]:
uf.in_range(meshx_gnd_da, [-2*dist*dx,2*dist*dx]).plot()

In [ ]:
meshx_gnd_da.plot()

In [ ]:
meshx_gnd_da.plot()

In [ ]:
uf.in_range(meshx_gnd_da, [-dx,dx]).plot()

In [ ]:
    dx = abs(meshx_gnd_da.x[2] - meshx_gnd_da.x[1])
    dy = abs(meshx_gnd_da.y[2] - meshx_gnd_da.y[1])
    
    max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
    half_range = round(max_len_xy/2)
    
    mask_core = mask_gnd.where(~((uf.in_range(meshx_gnd_da, [-2*dist*dx,2*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-2*dist*dy,2*dist*dy]))), 5)